# React `useContext` Hook and Context API 

## Introduction

This lecture explains the React Context API and the `useContext` hook by starting from the problem of prop drilling and then building practical examples, including a theme toggler and multiple contexts (user + theme). The main goals are to understand what prop drilling is, why it is problematic, and how `createContext`, Provider, and `useContext` work together to solve it in React applications. 

***

## Prop Drilling Problem

Prop drilling occurs when data from a parent component must be passed down through several intermediate components that do not actually need that data, just so a deeply nested child can finally use it. 

- Imagine:
  - A Parent component contains several nested children: C1, C2, C3, C4, C5, C6, C7 in a hierarchy. 
  - Parent has a prop that only component C5 needs. 
- Traditional props-based approach:
  - Parent → C3 → C4 → C5.
  - C3 and C4 receive the prop only to pass it further down, even though they never use it. 

Why prop drilling is bad:

- Intermediate components receive unnecessary data, increasing coupling between components. 
- Component trees in real apps are often large, so data may pass through many layers, making code harder to maintain and reason about. 
- Any change to the prop’s structure can force changes in multiple intermediate components that only forward the data. 

***

## Motivation for Context API

To avoid prop drilling, React provides Context, which allows data to be shared across a component tree without manually passing props through each level. 

Key idea:

- The component that has the data acts as a **Provider**.
- Any component that needs the data becomes a **Consumer**.
- Intermediate components that do not need the data are not forced to receive or forward it. 

Benefits:

- Cleaner component interfaces (fewer props purely for forwarding). 
- Easier to manage global or semi-global data like theme, user info, auth state, etc. 
- More scalable when the component hierarchy becomes deep. 

***

## Three/Four Core Steps of Using Context

The lecture explains the pattern as three main steps (often seen as four when splitting “use value + export”). 

### 1. Create Context

Use `createContext` to create a context object. 

- Example conceptual pattern:
  - `const UserContext = createContext();`. 
- This context object will later be used to:
  - Wrap children using `UserContext.Provider`.
  - Consume data using `useContext(UserContext)`. 

Important:

- Context can be created either in the parent component file or in a separate file dedicated to context. 

### 2. Wrap Children in a Provider

Use the created context’s `Provider` component to wrap all children that should be able to access the shared data. 

- Concept:
  - `UserContext.Provider` wraps ChildA.
  - ChildB is inside ChildA, and ChildC is inside ChildB.
  - Since ChildA is inside the Provider, all nested children (A, B, C) can become consumers if needed. 

Critical point:

- Any component that may need to consume the context value in the future must be inside the Provider’s subtree. 

### 3. Pass the Value into Provider

The Provider receives a `value` prop, which is the data you want to share. 

- Example pattern:
  - `value={user}` where `user` might be an object from `useState` (e.g., `{ name: "Love" }`). 
- Steps done so far:
  - Step 1: Create context.
  - Step 2: Wrap children in Provider.
  - Step 3: Pass `value` into Provider. 

### 4. Consume Context with `useContext`

Inside any consumer component (e.g., ChildC), use the `useContext` hook to read the data. 

- Conceptual pattern:
  - Import the context: `UserContext`.
  - `const user = useContext(UserContext);`. 
  - Use `user.name` (e.g., to render `Love`). 

Summary of steps:

1. Create context (e.g., `UserContext = createContext()`). 
2. Wrap children in `<UserContext.Provider>` in the component that has the data. 
3. Pass `value` to Provider (e.g., `value={user}` or `value={{ theme, setTheme }}`). 
4. In any descendant component, call `useContext(UserContext)` to consume the value. 

***

## Simple User Context Example

The lecture first builds a basic example with a `UserContext` to demonstrate one-way data flow without prop drilling. 

Component hierarchy:

- `App` (parent / top-level).
- `ChildA` inside `App`.
- `ChildB` inside `ChildA`.
- `ChildC` inside `ChildB`. 

Steps:

- In `App`:
  - Create `UserContext` with `createContext()`. 
  - Use `useState` to hold user data, e.g., `const [user, setUser] = useState({ name: "Love" });`. 
  - Wrap `<ChildA />` with `<UserContext.Provider value={user}>`. 
- In `ChildC`:
  - Import `UserContext`.
  - `const user = useContext(UserContext);`. 
  - Render `user.name`, which prints the user’s name coming directly from `App` without passing props through `ChildA` and `ChildB`. 

Key learning:

- Parent provides data via Provider.
- Deep child consumes data via `useContext`, skipping intermediate prop passing. 

***

## Theme Context Example with Toggling

The lecture then creates a more interesting project: a theme toggler using Context to show UI changes based on a global state (theme). 

### Component Setup

Hierarchy (same pattern):

- `App` (top-level).
- `ChildA` inside `App`.
- `ChildB` inside `ChildA`.
- `ChildC` inside `ChildB`. 

Goal:

- `App` holds `theme` state (`"light"` or `"dark"`).
- `ChildC` has a button to toggle the theme.
- A container’s background color changes based on the current theme. 

### Creating Theme Context and State

In `App`:

- Create context:
  - `const ThemeContext = createContext();`. 
- Create state:
  - `const [theme, setTheme] = useState("light");`. 

### Providing Theme Context

Wrap the target subtree with Provider:

- `<ThemeContext.Provider value={theme}>`
  - Place `<ChildA />` inside this provider. 
- This makes `theme` accessible in any nested component (ChildA, ChildB, ChildC). 

Later, to allow toggling, the value is changed to include both `theme` and `setTheme`:

- `value={{ theme, setTheme }}` so consumers can both read and update theme. 

### CSS and Container Setup

In `App`:

- Add a container `div` with id `container` and place `<ChildA />` inside it. 

In CSS:

- Style `#container`:
  - Height and width (e.g., 200px each).
  - Border like `1px solid black`.
  - `display: flex`, `flex-direction: column`.
  - `justify-content: center`, `align-items: center` to center content. 
  - Initial `background-color` set to something like beige. 

Later, the `background-color` will depend on the current `theme` (light or dark). 

### Consuming Theme and Toggling in ChildC

In `ChildC`:

1. Import `ThemeContext`.
2. Use `useContext`:
   - Initially: `const theme = useContext(ThemeContext);` to read theme. 
   - After adding `setTheme`, destructure both:
     - `const { theme, setTheme } = useContext(ThemeContext);`. 
3. Add a button:
   - Label: e.g., “Change Theme”. 
   - Attach `onClick` to a handler function.

Handler logic:

- In `ChildC`, define a `handleClick` (or `toggleTheme`) that switches between `"light"` and `"dark"` using `setTheme`:
  - If `theme === "light"`, set it to `"dark"`.
  - Else set it back to `"light"`. 

### Applying Theme to UI

Back in `App`:

- Use inline style on the container `div`:

  - Background color depends on `theme`:
    - If `theme === "light"`, use beige (or some light color).
    - Else use black. 

Effect:

- Initially, theme is `"light"`, so background is light/beige. 
- Clicking “Change Theme”:
  - Toggles `theme` to `"dark"`.
  - The container background becomes black. 
- Clicking again toggles back to `"light"`, background returns to light. 

Conceptual points:

- Context makes both `theme` and `setTheme` available deep in the tree.
- The deep child can change global UI state without prop drilling. 

***

## Using Multiple Contexts Together

The lecture also demonstrates that multiple contexts can coexist in the same parent component. 

Contexts:

- `UserContext` (with example user data, e.g., `{ name: "Love" }`). 
- `ThemeContext` (with `theme` and `setTheme`). 

In `App`:

- Both contexts are created using `createContext()`. 
- User data is stored via `useState` and provided via `UserContext.Provider`. 
- Theme data/state is provided via `ThemeContext.Provider`. 

Provider nesting:

- One provider wraps the other:
  - For example:
    - Outer: `<UserContext.Provider value={user}>`
    - Inner: `<ThemeContext.Provider value={{ theme, setTheme }}>`
    - Inside: `<ChildA />`. 

Effect:

- Any nested component (like `ChildC`) now has access to both user and theme contexts. 

In `ChildC`:

- Import both contexts: `UserContext`, `ThemeContext`. 
- Use them:
  - `const { theme, setTheme } = useContext(ThemeContext);`
  - `const user = useContext(UserContext);`. 
- Use `user.name` to display user info and `theme`/`setTheme` for theme toggling. 

Key learning:

- React supports multiple independent contexts.
- Providers can be nested, and each provider grants access to its value for all descendants. 

***

## Important Terms and Concepts

- **Context**: A mechanism in React for passing data through the component tree without manually passing props at every level. 
- **`createContext()`**: Function used to create a new context object. 
- **Provider**:
  - A component (e.g., `UserContext.Provider`) that makes a value available to all its descendants. 
  - Accepts a `value` prop containing the shared data (state, functions, objects, etc.). 
- **Consumer / `useContext`**:
  - Consumers are components that call `useContext(SomeContext)` to read the value. 
  - `useContext` subscribes to the nearest matching Provider above in the tree. 
- **Prop Drilling**:
  - Passing props down multiple levels just to reach a deep child, even when intermediate components do not need the data. 

***

## Practical Patterns to Remember

When using Context and `useContext`, follow this mental checklist:

1. **Create the context**:
   - `const SomeContext = createContext();`. 
2. **Create the data (possibly state)**:
   - For static data: define variables/objects.
   - For dynamic data: use `useState`, `useReducer`, etc. 
3. **Wrap the relevant subtree in Provider and pass value**:
   - `<SomeContext.Provider value={valueToShare}>`.
   - Place all components that need access inside this Provider. 
4. **Export the context**:
   - `export default SomeContext;` or `export { SomeContext };` so other components can import it. 
5. **Consume using `useContext` in any descendant**:
   - `const data = useContext(SomeContext);`. 
   - Use `data` in JSX or logic as required (render, toggle, etc.). 

***

## Summary of Main Takeaways

- Prop drilling is the practice of passing data through many components that do not need it, just to reach a nested component, which makes code messy and harder to maintain. 
- React Context API, combined with the `useContext` hook, solves prop drilling by letting a Provider component share data with any descendant Consumer without intermediate props. 
- The typical workflow is:
  1. Create context with `createContext()`.
  2. Wrap needed components in the context’s Provider.
  3. Pass data via the Provider’s `value` prop.
  4. Use `useContext` in descendants to consume that value. 
- A theme toggler example shows how a deep child can both read and update global theme state and how UI (like container background) reacts to theme changes. 
- Multiple contexts (e.g., `UserContext` and `ThemeContext`) can be used together, with nested Providers, enabling components to access multiple independent pieces of shared state without prop drilling. 
